# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [35]:

# read the Cities_weather_data.csv to create dataframe, previous cells are not needed to be executed as long as Cities_weather_data.csv was created
cities = pd.read_csv("Output_data/Cities_weather_data.csv", encoding="utf-8")
cities.head()

,City,Latitude,Longitudes,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Port Lincoln,-34.73,135.87,54.28,67,36,53.44,AU,1600698584
1,Port Hardy,50.70,-127.42,51.80,93,90,2.39,CA,1600698585
2,Beringovskiy,63.05,179.32,40.03,90,100,23.96,RU,1600698585
3,Yanji,42.91,129.51,51.19,81,82,2.39,CN,1600698585
4,Kangaatsiaq,68.31,-53.46,34.43,86,100,3.89,GL,1600698585


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
humidity = cities["Humidity (%)"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Latitude", "Longitudes"]]

In [39]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
new_city_df = cities.loc[(cities["Max Temp (F)"] > 70) & (cities["Max Temp (F)"] < 80) & (cities["Wind Speed (mph)"] < 10), :]
new_city_df = new_city_df.dropna(how='any')
new_city_df.reset_index(inplace=True)
del new_city_df['index']
new_city_df.head()
len(new_city_df)

39

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
hotellist = []

for i in range(len(new_city_df)):
    lat = new_city_df.loc[i]['Latitude']
    lng = new_city_df.loc[i]['Longitudes']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    json_data = response.json()
    try:
        hotellist.append(json_data['results'][0]['name'])
    except:
        hotellist.append("")
new_city_df["Hotel Name"] = hotellist
new_city_df = new_city_df.dropna(how='any')
new_city_df.head()
len(new_city_df)

39

In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_city_df.iterrows()]
locations = new_city_df[["Latitude", "Longitudes"]]

In [33]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))